In [5]:
import numpy as np

import pickle
import time
import datetime

from make_test_samples import *

import matplotlib.pyplot as plt
import seaborn as sns; sns.set()


C:\ProgramData\Anaconda3\envs\temporal_coding\lib\site-packages\brian2\core\variables.py:174: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  return np.issubdtype(np.bool, self.dtype)


# Data generation

In [64]:
# Global generation parameters
set_size = 100
num_neur= 30
rate = 15
duration_ms = 500

# Paramters for generation using temporal shift
tempshift_params = dict(
    rate=rate,
    num_neur=num_neur,
    duration_ms=duration_ms,
    shift_size=5,
    set1_size=set_size,
    set2_size=set_size)

# Parameters for generation using simplified vesicle release model
vesrel_params = dict(
    rate=rate,
    num_neur=num_neur,
    duration_ms=duration_ms,
    span=5,
    mode=1,
    num_ves=20,
    set1_size=set_size, 
    set2_size=set_size)

# Generate samples
all_samples_temporal_shift = gen_with_temporal_shift(**tempshift_params)
all_samples_vesicle_release = gen_with_vesicle_release(**vesrel_params)

# Isolate single samples for use in testing
tempshift_num = all_samples_temporal_shift['data'][1]
vesrel_num = all_samples_vesicle_release['data'][1]

WARNING    C:\ProgramData\Anaconda3\envs\temporal_coding\lib\site-packages\brian2\core\variables.py:174: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  return np.issubdtype(np.bool, self.dtype)
 [py.warnings]
WARNING    C:\ProgramData\Anaconda3\envs\temporal_coding\lib\site-packages\brian2\core\variables.py:174: FutureWarning: Conversion of the second argument of issubdtype from `bool` to `np.generic` is deprecated. In future, it will be treated as `np.bool_ == np.dtype(bool).type`.
  return np.issubdtype(np.bool, self.dtype)
 [py.warnings]
WARNING    C:\ProgramData\Anaconda3\envs\temporal_coding\lib\site-packages\brian2\groups\group.py:258: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int32 == np.dtype(int).type`.
  elif not np.issubdtype(index_array.d

Vectorization function to normalize sample times to matrices

In [79]:
def vectorize_sample(sample, duration_ms, dt=0.1):
    # Assumes sample times are given in ms
    num_trains = sample.shape[0]
    num_bins = int(duration_ms / dt)
    binarized = np.zeros((num_trains, num_bins))
    for i,train in enumerate(sample):
        times = (train/dt).astype(int)
        inds, counts = np.unique(times, return_counts=True)
        binarized[i][inds] = counts
    return binarized


Flattening function for testing on sequential vectors

In [77]:
def flatten_sample(sample):
    inds = []
    times = []
    for i, neuron in enumerate(sample):
        inds.extend([i] * neuron.shape[0])
        times.extend(neuron)
    return np.array([inds, times])

Generate samples for testing

In [82]:
dt = 0.1
tempshift_bin = vectorize_sample(tempshift_num, duration_ms, dt)
vesrel_bin = vectorize_sample(vesrel_num, duration_ms, dt)
tempshift_flat = flatten_sample(tempshift_num)
vesrel_flat = flatten_sample(vesrel_num)

# Model testing

In [72]:
from numba import jit

${dv\over dt}  = -{v\over \tau}$